In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Load the CSV data into a DataFrame
df = pd.read_csv('erc20_[20].csv')

In [ ]:
# Summary statistics
print("\nSummary Statistics:")
print(df.describe())

In [ ]:
# Count of verified vs. not verified
print("\nCount of Verified vs. Not Verified:")
print(df['verified'].value_counts())

In [ ]:
# Average time taken for verified and not verified runs
print("\nAverage Time Taken for Verified and Not Verified Runs (minutes:seconds):")
print(df.groupby('verified')['time_taken'].mean())

In [ ]:
# Number of iterations for verified and not verified runs
print("\nAverage Iterations for Verified and Not Verified Runs:")
print(df.groupby('verified')['iterations'].mean().round())

In [ ]:
def parse_status_logs(log):
    parsed_logs = {}
    if log:
        try:
            # Manually split the logs and process each entry
            log_list = log.strip('[]').split('", "')
            for entry in log_list:
                interaction_logs = entry.split('\\n')
                interaction_number = None
                for interaction_log in interaction_logs:
                    if 'Iteraction:' in interaction_log:
                        interaction_number = int(interaction_log.split(': ')[1])
                        if interaction_number not in parsed_logs:
                            parsed_logs[interaction_number] = {}
                    if 'OK' in interaction_log or 'ERROR' in interaction_log:
                        parts = interaction_log.split(': ')
                        method = parts[0].split('::')[1]
                        status = parts[1].strip()
                        if interaction_number is not None:
                            parsed_logs[interaction_number][method] = status
        except Exception as e:
            print(f"Error parsing log: {e}")
    return parsed_logs

# Apply the parse_status_logs function to each row's status column
df['parsed_status'] = df['status'].apply(parse_status_logs)

print("\nParsed Status:")
print(df['parsed_status'])


In [ ]:
# Create a dictionary to hold the data for plotting
plot_data = {}

# Extract data for plotting
for parsed_log in df['parsed_status']:
    for interaction, methods in parsed_log.items():
        if interaction not in plot_data:
            plot_data[interaction] = {}
        for method, status in methods.items():
            if method not in plot_data[interaction]:
                plot_data[interaction][method] = status

# Create a DataFrame from the plot data
plot_df = pd.DataFrame.from_dict(plot_data, orient='index')

# Convert 'OK'/'ERROR' to numerical values (OK = 1, ERROR = 0)
plot_df = plot_df.applymap(lambda x: 1 if x == 'OK' else 0)

# Sort the DataFrame by interaction numbers
plot_df = plot_df.sort_index()

# Plot a heatmap
plt.figure(figsize=(14, 10))
sns.heatmap(plot_df.T, cmap='coolwarm', cbar_kws={'ticks': [0, 1]}, annot=True, fmt='d')
plt.xlabel('Interaction Number')
plt.ylabel('Function')
plt.title('Evolution of Function Status Over Interactions')
plt.xticks(rotation=45)
plt.yticks(rotation=0)
plt.show()

In [ ]:
# Plot the distribution of time taken
plt.figure(figsize=(10, 5))
df['time_taken'].hist(bins=10)
plt.title('Distribution of Time Taken')
plt.xlabel('Time Taken (seconds)')
plt.ylabel('Frequency')
plt.show()

In [ ]:
# Plot the distribution of iterations
plt.figure(figsize=(10, 5))
df['iterations'].hist(bins=10)
plt.title('Distribution of Iterations')
plt.xlabel('Iterations')
plt.ylabel('Frequency')
plt.show()

In [ ]:
# Plot the average time taken for verified vs. not verified runs
avg_time_taken = df.groupby('verified')['time_taken'].mean()
print(avg_time_taken)

In [ ]:
# Plot the average iterations for verified vs. not verified runs
avg_iterations = df.groupby('verified')['iterations'].mean()
print(avg_iterations)